In [86]:
import pandas as pd
import numpy as np
import glob
from bs4 import BeautifulSoup, Comment
import csv
import requests
from unidecode import unidecode
from datetime import datetime
import time
import random

### Web scrapping stadium data from wikipedia page

In [22]:
urls = ["https://en.wikipedia.org/wiki/2014–15_La_Liga",
        "https://en.wikipedia.org/wiki/2015–16_La_Liga",
        "https://en.wikipedia.org/wiki/2016–17_La_Liga",
        "https://en.wikipedia.org/wiki/2017–18_La_Liga",
        "https://en.wikipedia.org/wiki/2018–19_La_Liga",
        "https://en.wikipedia.org/wiki/2019–20_La_Liga",
        "https://en.wikipedia.org/wiki/2020–21_La_Liga",
        "https://en.wikipedia.org/wiki/2021–22_La_Liga",
        "https://en.wikipedia.org/wiki/2022–23_La_Liga",
        "https://en.wikipedia.org/wiki/2023–24_La_Liga",
        "https://en.wikipedia.org/wiki/2024–25_La_Liga"]

data = []

for url in urls:
    season = int(url.split("_")[0].split("/")[-1].split('–')[0])

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    table = soup.find('table',{'class':'wikitable sortable'})    
    rows = table.find_all('tr')[1:]
    
    for row in rows:
        cols = row.find_all('td')
        if len(cols) >= 3:
            team = cols[0].text.strip()
            city = cols[1].text.strip()
            stadium = cols[2].text.strip()
            data.append({
                'Season': season,
                'Team': team,
                'Stadium': stadium,
                'City': city})

stadiums = pd.DataFrame(data)

### Using the MapBox API to add latitude and longitude to the stadiums data.

In [27]:
MAPBOX_API_KEY = "pk.eyJ1Ijoic2hhdW5qb3NlIiwiYSI6ImNtNzhhOXp6MDFqb2Iyb3BwNnFxcXVxNW4ifQ.QhBwaGVLhveGqi7Ui926kA"

def geocode_with_mapbox(location_query):
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{location_query}.json"
    params = {
        'access_token': MAPBOX_API_KEY,
        'country': 'es',  # Limit to Spain
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if data['features']:
            location = data['features'][0]
            latitude = location['center'][1]
            longitude = location['center'][0]
            country = location['context'][-1]['text']
            place = location['context'][1]['text']
            
            return latitude, longitude, place, country

def add_lat_long(row):

    location_query = f"{row['Stadium']}, {row['City']}"
    
    result = geocode_with_mapbox(location_query)
    
    if result is not None:
        latitude, longitude, place, country = result

        row['Latitude'] = latitude
        row['Longitude'] = longitude
        row['Place'] = place
        row['Country'] = country
    else:

        row['Latitude'] = None
        row['Longitude'] = None
        row['Place'] = None
        row['Country'] = None
    
    return row

stadium_lat_long = stadiums.apply(add_lat_long, axis=1)

stadium_lat_long = stadiums.apply(add_lat_long, axis=1)


In [ ]:
for col in stadium_lat_long.columns:
    if stadium_lat_long[col].isna().sum() > 0:
        mode_value = stadium_lat_long[stadium_lat_long['Team'] == 'Real Sociedad'][col].mode()[0]
        stadium_lat_long[col].fillna(mode_value, inplace=True) 
stadium_lat_long[stadium_lat_long['Team'] == 'Real Sociedad'].tail()

In [88]:
# Changing the Team name to unicode
stadium_lat_long['Team_unicode'] = stadium_lat_long['Team'].apply(unidecode)
stadium_lat_long.drop(['Team'], axis =1 ,inplace = True)

### Importing the downloaded match files from datahub.io and combining them to a single csv file

In [90]:
# Path to the folder containing CSV files
folder_path = 'C:/Users/shaun/Laliga/season*.csv'

# Use glob to get all CSV files in the folder
all_files = glob.glob(folder_path)

# Read and concatenate all CSV files
matches_combined = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

# Save the combined DataFrame to a new CSV file
matches_combined.to_csv('C:/Users/shaun/Laliga/Final_match_data/matches_combined.csv', index=False)

In [94]:
# There are some mismatches in the team names between Team_unicode and HomeTeam.
stadium_lat_long.replace({'Team_unicode' : { 'Athletic Bilbao' : 'Ath Bilbao', 'Atletico Madrid' : 'Ath Madrid', 'Celta Vigo' : 'Celta',
                                           'Espanyol': 'Espanol','Rayo Vallecano':'Vallecano','Real Betis':'Betis', 'Sporting Gijon':'Sp Gijon',
                                            'Deportivo La Coruna':'La Coruna','Real Sociedad':'Sociedad'}}, inplace = True)

### Merging Match data with Stadium data

In [69]:
matches_stadiums_merge = pd.merge(matches_combined, stadium_lat_long, how='left', left_on=['Season', 'HomeTeam'], right_on=['Season', 'Team_unicode'])

In [71]:
# Creating a function to make the date format consisitent across all the rows
def convert_two_digit_year(date):
    try:
        return pd.to_datetime(date, format='%d/%m/%y', dayfirst=True).strftime('%d/%m/%Y')
    except Exception:
        return pd.to_datetime(date, dayfirst=True).strftime('%d/%m/%Y')

matches_stadiums_merge["Date"] = matches_stadiums_merge["Date"].apply(convert_two_digit_year)

### Fetching weather data using Open-Meteo API

In [76]:
import pandas as pd
import requests
from datetime import datetime

def get_weather_data(latitude, longitude, date):

    api_url = "https://archive-api.open-meteo.com/v1/archive"

    date_obj = datetime.strptime(date, '%d/%m/%Y')
    formatted_date = date_obj.strftime('%Y-%m-%d')

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'start_date': formatted_date,
        'end_date': formatted_date,
        'daily': ['temperature_2m_max','temperature_2m_min','apparent_temperature_max','apparent_temperature_min',
                  'windspeed_10m_max','weather_code', 'precipitation_sum','rain_sum','showers_sum','snowfall_sum',
                 'precipitation_probability_mean'],
        'timezone': 'auto' 
    }

    response = requests.get(api_url, params=params)
    response.raise_for_status() 
    data = response.json()

    daily_data = data.get('daily', {})
      
    max_temp = daily_data['temperature_2m_max'][0]
    min_temp = daily_data['temperature_2m_min'][0]
    apparent_temperature_max = daily_data['apparent_temperature_max'][0]
    apparent_temperature_min = daily_data['apparent_temperature_min'][0]
    wind_speed = daily_data['windspeed_10m_max'][0]
    weather_code = daily_data['weather_code'][0]
    precipitation_sum = daily_data['precipitation_sum'][0]
    rain_sum = daily_data['rain_sum'][0]
    showers_sum = daily_data['showers_sum'][0]
    snowfall_sum = daily_data['snowfall_sum'][0]
    precipitation_probability_mean = daily_data['precipitation_probability_mean'][0]

    return {
        'max_temp': max_temp,
        'min_temp': min_temp,
        'apparent_temperature_max': apparent_temperature_max,
        'apparent_temperature_min': apparent_temperature_min,
        'wind_speed': wind_speed,
        'weather_code' : weather_code,
        'precipitation_sum': precipitation_sum,
        'rain_sum':rain_sum,
        'showers_sum':showers_sum,
        'snowfall_sum':snowfall_sum,
        'precipitation_probability_mean': precipitation_probability_mean}
        
weather_data_list = []

for index, row in matches_stadiums_merge.iterrows():
    date = row['Date']
    latitude = row['Latitude']
    longitude = row['Longitude']

    weather_data = get_weather_data(latitude, longitude, date)

    weather_data["Date"] = row["Date"]
    weather_data["Stadium"] = row["Stadium"]
    weather_data_list.append(weather_data)
    print(index)
    
weather = pd.DataFrame(weather_data_list)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [84]:
final_df = pd.merge(matches_stadiums_merge, weather, on=['Date','Stadium',], how='left')

### Scrapping transfermarkt website to get valuation of each team for all the seasons

In [101]:
base_url = "https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id={}"
seasons = [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.google.com/'}

all_seasons_data = []

for season in seasons:
    url = base_url.format(season)
    print(f"Scraping data from: {url}")

    # Send request with headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', class_='items')

    for row in table.find_all('tr', class_=['odd', 'even']):
        cells = row.find_all('td')
        team_name = cells[1].find('a').text.strip() 
        squad = cells[2].text.strip()     
        avg_age = cells[3].text.strip()            
        foreigners = cells[4].text.strip()         
        avg_market_value = cells[5].text.strip()   
        total_market_value = cells[6].text.strip()

        all_seasons_data.append([season, team_name, squad, avg_age, foreigners, avg_market_value, total_market_value])
        
    time.sleep(random.uniform(1, 5))

club_valuation = pd.DataFrame(all_seasons_data, columns=['Season', 'Team', 'Squad', 'Avg Age', 'Foreigners', 'Avg Market Value', 'Total Market Value'])

Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2014
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2015
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2016
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2017
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2018
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2019
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2020
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2021
Scraping data from: https://www.transfermarkt.com/la-liga/startseite/wettbewerb/ES1/plus/?saison_id=2022
Scraping data from: https://www.transfermarkt.com/la-li

In [ ]:
club_valuation['Team'] = club_valuation['Team'].apply(unidecode)

In [ ]:
club_valuation.replace({'Team' : { 'Athletic Bilbao' : 'Ath Bilbao', 'Atletico de Madrid' : 'Ath Madrid','Deportivo Alaves' : 'Alaves',
                                  'FC Barcelona':'Barcelona','UD Almeria' : 'Almeria', 'Celta de Vigo' : 'Celta', 'CA Osasuna': 'Osasuna', 
                                  'Cadiz CF':'Cadiz','Elche CF':'Elche', 'Getafe CF':'Getafe', 'Girona FC':'Girona','Granada CF':'Granada', 
                                  'Levante UD':'Levante','RCD Mallorca':'Mallorca', 'Real Valladolid CF':'Valladolid', 'SD Eibar':'Eibar', 
                                  'SD Huesca':'Huesca','Sevilla FC':'Sevilla', 'UD Las Palmas':'Las Palmas', 'Valencia CF':'Valencia',
                                  'Villarreal CF':'Villarreal','RCD Espanyol Barcelona': 'Espanol','Rayo Vallecano':'Vallecano',
                                  'Real Betis Balompie':'Betis','Real Sociedad':'Sociedad','Sporting Gijon':'Sp Gijon', 'Cordoba CF':'Cordoba',
                                  'Deportivo de La Coruna':'La Coruna', 'CD Leganes':'Leganes', 'Malaga CF':'Malaga'}}, inplace = True)

In [ ]:
df = pd.merge(final_df, club_valuation,how='left', left_on=['Season','HomeTeam'], right_on=['Season','Team'])

In [ ]:
df.drop(['Unnamed: 0'], axis =1,inplace = True)

In [ ]:
df.to_csv('C:/Users/shaun/Laliga/final_weather_match_valuation.csv')

In [ ]:
# Key learning point was debugging to overcome the issue of tables being dynamically generated and embedded   with HTML comments.
''' Issue: The table we are trying to scrape is dynamically generated and embedded within HTML comments in FBref. It makes the table invisible to
standard HTML parsers like BeautifulSoup. 
Solution: To handle this, we need to extract and parse the commented-out HTML content'''

### Web scrapping the FBref website to get players data

In [56]:
base_url = "https://fbref.com/en/comps/12/{season}/stats/{season}-La-Liga-Stats"

seasons = ["2014-2015", "2015-2016", "2016-2017"]

all_seasons_data = []

def scrape_season(season):
    url = base_url.format(season=season)
    response = requests.get(url)
    response.raise_for_status()
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract comments containing HTML as FBref tables are often within comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    
    # Find the table in the commented-out HTML
    table_html = None
    for comment in comments:
        if 'stats_standard' in comment:
            table_html = BeautifulSoup(comment, 'html.parser').find('table', {'id': 'stats_standard'})
            break
    
    if not table_html:
        raise ValueError(f"The specified table with id 'stats_standard' could not be found for season {season}.")
    
    headers = [th.text.strip() for th in table_html.find('thead').find_all('th')]
    rows = table_html.find('tbody').find_all('tr', class_=lambda x: x != "thead")

    player_data = []
    for row in rows:
        if row.find('td'):  # Ensure it's a valid data row
            cols = [col.text.strip() for col in row.find_all('td')]
            player_data.append(cols[:-1])
    
    new_headers = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 
                   'CrdY', 'CrdR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK']

    df = pd.DataFrame(player_data, columns=new_headers)
    season_year = season.split("-")[0]
    df.insert(0, 'Season', season_year)
    return df

for season in seasons:
    season_df = scrape_season(season)
    all_seasons_data.append(season_df)
    print(f"Data for season {season} scraped successfully.")

players = pd.concat(all_seasons_data, ignore_index=True)

Data for season 2014-2015 scraped successfully.
Data for season 2015-2016 scraped successfully.
Data for season 2016-2017 scraped successfully.


In [58]:
players14_16 = players

In [70]:
base_url = "https://fbref.com/en/comps/12/{season}/stats/{season}-La-Liga-Stats"

seasons = ["2017-2018", "2018-2019", "2019-2020","2020-2021", "2021-2022", "2022-2023", "2023-2024", "2024-2025"]

all_seasons_data = []

def scrape_season(season):
    url = base_url.format(season=season)
    
    # Send a GET request to fetch the HTML content
    response = requests.get(url)
    response.raise_for_status()
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract comments containing HTML as FBref tables are often within comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    
    # Find the table in the commented-out HTML
    table_html = None
    for comment in comments:
        if 'stats_standard' in comment:
            table_html = BeautifulSoup(comment, 'html.parser').find('table', {'id': 'stats_standard'})
            break
    
    if not table_html:
        raise ValueError(f"The specified table with id 'stats_standard' could not be found for season {season}.")
    
    # Extract headers dynamically from the table
    headers = [th.text.strip() for th in table_html.find('thead').find_all('th')]
    
    # Extract player rows (avoid subheaders or empty rows)
    rows = table_html.find('tbody').find_all('tr', class_=lambda x: x != "thead")
    
    # Parse player data into a list of lists
    player_data = []
    for row in rows:
        if row.find('td'):  # Ensure it's a valid data row
            cols = [col.text.strip() for col in row.find_all('td')]
            player_data.append(cols[:-1]) #Removes the last column
    
    new_headers = ['Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 
                   'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls', 'Ast', 'G+A', 'G-PK', 'G+A-PK', 'xG', 'XAG',
                   'XG+AG', 'npxG', 'npxG+AG']

    df = pd.DataFrame(player_data, columns=new_headers)
    season_year = season.split("-")[0]
    df.insert(0, 'Season', season_year)
    return df

for season in seasons:
    season_df = scrape_season(season)
    all_seasons_data.append(season_df)
    print(f"Data for season {season} scraped successfully.")

players = pd.concat(all_seasons_data, ignore_index=True)

Data for season 2017-2018 scraped successfully.
Data for season 2018-2019 scraped successfully.
Data for season 2019-2020 scraped successfully.
Data for season 2020-2021 scraped successfully.
Data for season 2021-2022 scraped successfully.
Data for season 2022-2023 scraped successfully.
Data for season 2023-2024 scraped successfully.
Data for season 2024-2025 scraped successfully.


In [72]:
players_17_24 = players

In [76]:
players14_16.to_csv('C:/Users/shaun/Laliga/Players/play_14_16.csv')
players_17_24.to_csv('C:/Users/shaun/Laliga/Players/play_17_24.csv')

In [80]:
df_players = pd.read_csv('C:/Users/shaun/Laliga/Players/final_players_data.csv')